# Simple Classification Model

In [9]:
from river import metrics, datasets, compose, preprocessing
from deep_river.classification import Classifier
from torch import nn
from tqdm import tqdm
from pprint import pprint

In [10]:
datasets.Phishing()

Phishing websites.

This dataset contains features from web pages that are classified as phishing or not.

    Name  Phishing                                                                                                            
    Task  Binary classification                                                                                               
 Samples  1,250                                                                                                               
Features  9                                                                                                                   
  Sparse  False                                                                                                               
    Path  /Users/kulbach/Documents/environments/deep-river-demo-23/lib/python3.11/site-packages/river/datasets/phishing.csv.gz

In [12]:
for x, y in dataset:
    pprint(x)
    print(f'Phishing: {y}')
    break

{'age_of_domain': 1,
 'anchor_from_other_domain': 0.0,
 'empty_server_form_handler': 0.0,
 'https': 0.0,
 'ip_in_url': 1,
 'is_popular': 0.5,
 'long_url': 1.0,
 'popup_window': 0.0,
 'request_from_other_domain': 0.0}
Phishing: True


In [2]:
dataset = datasets.Phishing()
metric = metrics.Accuracy()

class MyModule(nn.Module):
    def __init__(self, n_features):
        super(MyModule, self).__init__()
        self.dense0 = nn.Linear(n_features,5)
        self.nonlin = nn.ReLU()
        self.dense1 = nn.Linear(5, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.nonlin(self.dense1(X))
        X = self.softmax(X)
        return X

model_pipeline = compose.Pipeline(
    preprocessing.StandardScaler,
    Classifier(module=MyModule,loss_fn="binary_cross_entropy",optimizer_fn='adam')
)
model_pipeline

Pipeline (
  StandardScaler (
    with_std=True
  ),
  Classifier (
    module=None
    loss_fn="binary_cross_entropy"
    optimizer_fn=<class 'torch.optim.adam.Adam'>
    lr=0.001
    output_is_logit=True
    is_class_incremental=False
    device="cpu"
    seed=42
  )
)

In [3]:
for x,y in tqdm(dataset.take(5000)):
    y_pred = model_pipeline.predict_one(x)      # make a prediction
    metric = metric.update(y, y_pred)  # update the metric
    model_pipeline = model_pipeline.learn_one(x, y)    # make the model learn
print(f'Accuracy: {metric.get()}')

1250it [00:01, 1211.49it/s]

Accuracy: 0.6728
